# Interpolating file sequences

For cases in which interpolation is needed, one must interpolate a series of files. Here we show how that can be done easily.

#### Import general modules

In [1]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np

# Get mpi info
comm = MPI.COMM_WORLD

#### Import modules from pynektools

In [2]:
from pynektools.interpolation.wrappers import interpolate_fields_from_disk

## Interpolate fields from a list


First we will define the inputs. For this, we need the name of the file that has the points, the name of the file that contains the spectral element mesh and optionally the name of the outputs. The latter defaults to intepolated_fields.hdf5

In [3]:
query_points_fname = "points.hdf5"
data_path = "../data/sem_data/instantaneous/cylinder_rbc_nelv_600/"
sem_mesh_fname = f"{data_path}field0.f00801"
interpolated_fields_output_fname = "interpolated_fields.hdf5"

Additionally, we need a dictionary that we call settings. Here we give the information about where the fields will be read from.

We can provide as an input the file name sequence to be read by the interpolator. Or one can provide a file index produces by the indexing module of pynektools.

Lastly we must say which fields to interpolate. If one passes the list with the string "all", all fields in the file are interpolated.

In [4]:
# Comment the option that you do not want to use

# Option 1 (if you have created an index file before)
field_interpolation_dictionary = {}
field_interpolation_dictionary['input_type'] = "file_index"
field_interpolation_dictionary['file_index'] = f"{data_path}field.fld_index.json"
field_interpolation_dictionary['fields_to_interpolate'] = ["all"]

# Option 2 directly provide the names of the sequence of files you want to interpolate
#field_interpolation_dictionary = {}
#field_interpolation_dictionary['input_type'] = "file_sequence"
#field_interpolation_dictionary['file_sequence'] = [f"{data_path}field0.f{str(i).zfill(5)}" for i in range(801, 850)]
#field_interpolation_dictionary['fields_to_interpolate'] = ["all"]

In [5]:
interpolate_fields_from_disk(comm, query_points_fname, sem_mesh_fname, field_interpolation_dictionary, interpolated_fields_output_fname=interpolated_fields_output_fname)

2024-09-18 15:58:31,363 - Probes - INFO - Initializing Probes object
2024-09-18 15:58:31,365 - Probes - INFO - Reading probes from points.hdf5
2024-09-18 15:58:31,367 - Probes - WARNING - Warning: xyz not found in points.hdf5
2024-09-18 15:58:31,368 - Probes - WARNING - Attemping to use x,y,z coordinates
2024-09-18 15:58:31,469 - Probes - INFO - Input probes are distributed: False
2024-09-18 15:58:31,470 - Probes - INFO - Reading mesh from ../data/sem_data/instantaneous/cylinder_rbc_nelv_600/field0.f00801
2024-09-18 15:58:31,471 - Mesh - INFO - Initializing empty Mesh object.
2024-09-18 15:58:31,472 - pynekread - INFO - Reading file: ../data/sem_data/instantaneous/cylinder_rbc_nelv_600/field0.f00801
2024-09-18 15:58:31,479 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2024-09-18 15:58:31,480 - Mesh - INFO - Initializing common attributes.
2024-09-18 15:58:31,481 - Mesh - INFO - Getting vertices
2024-09-18 15:58:31,482 - Mesh - INFO - Getting edge centers
2024-09-18 15:5